<a href="https://colab.research.google.com/github/Shallom12/Autonomous-driving-of-Chungnam-Human-Resources-Development-Institute/blob/main/autonomous_driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this is a commet
print('hello world')

hello world


In [ ]:
x = 12
y = 'jeongjueun'
print(x)
print(y)
print(str(x)+y)

12
jeongjueun
12jeongjueun


In [ ]:
if 5 > 2:
  print('Five is greater than two')
x = 4
x = 'Daniel'
print(x)

Five is greater than two
Daniel


In [ ]:
x = str(3)
y = int(5)
z = float(7)
print(type(x))
print(type(y))
my_var_name = 'John'

<class 'str'>
<class 'int'>


In [ ]:
x, y, z= 'Orange', 'Banana', 'Cherry'
print(x)
print(y)
print(z)

Orange
Banana
Cherry


#python variables

In [ ]:
x = y = z = 'Orange'
print(x)
print(y)
print(x)

Orange
Orange
Orange


In [ ]:
fruits = ['apple','banana', 'cherry']
x, y, z = fruits
print(x)
print(y)
print(z)

apple
banana
cherry


In [ ]:
x = 'python'
y = 'is'
z = 'awesome'
print(x,y,z)
print(x+y+z)

python is awesome
pythonisawesome


In [ ]:
x = 5
y = 15
print(x+y)

20


In [ ]:
x = 'awesome'
def myfunc():
  print('python is' + x)
myfunc()
print('python is' + x)

python isawesome
python isawesome


### 1. 동영상 파일 불러오기 또는 웹캠 연결 (0번은 기본 카메라)
### 2. 이미지를 그레이스케일로 변환 (색 정보 제거, 연산 빠르게 하기 위함)
### 3. 가우시안 블러 적용 (노이즈 제거)
### 4. Canny 엣지 검출 (경계선 찾기)
### 5. 관심영역(ROI) 설정 - 삼각형 영역만 추출 (차선이 위치할 구간만 집중)
### 6. 허프 변환으로 직선 검출
### 7. 검출된 직선을 원본 이미지에 그리기
### 8. 원본 프레임과 직선 이미지 합성
### 9. 결과 출력
### 10. q 키를 누르면 종료

In [ ]:
import cv2
import numpy as np

# 1. 동영상 파일 불러오기 또는 웹캠 연결 (0번은 기본 카메라)
cap = cv2.VideoCapture("lane_video.mp4")  # 또는 cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()  # 한 프레임씩 읽어옴
    if not ret:
        break

    # 2. 이미지를 그레이스케일로 변환 (색 정보 제거, 연산 빠르게 하기 위함)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 3. 가우시안 블러 적용 (노이즈 제거)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # 4. Canny 엣지 검출 (경계선 찾기)
    edges = cv2.Canny(blur, 50, 150)

    # 5. 관심영역(ROI) 설정 - 삼각형 영역만 추출 (차선이 위치할 구간만 집중)
    height = frame.shape[0]
    polygon = np.array([[
        (0, height),
        (frame.shape[1], height),
        (frame.shape[1]//2, height//2)
    ]])
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, polygon, 255)
    cropped_edges = cv2.bitwise_and(edges, mask)

    # 6. 허프 변환으로 직선 검출
    lines = cv2.HoughLinesP(
        cropped_edges,
        rho=1,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=50,
        maxLineGap=150
    )

    # 7. 검출된 직선을 원본 이미지에 그리기
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # 8. 원본 프레임과 직선 이미지 합성
    combined = cv2.addWeighted(frame, 0.8, line_image, 1, 1)

    # 9. 결과 출력
    cv2.imshow("Lane Detection", combined)

    # 10. q 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


# 🎯 YOLOv5로 객체 탐지

Ultralytics에서 제공하는 `yolov5` 라이브러리를 이용해, 샘플 이미지를 대상으로 실시간 객체 인식을 실행해봅니다.

- **라이브러리**: yolov5 (PyTorch 기반)
- **샘플 이미지**: Zidane 축구경기 사진


In [ ]:
# 1️⃣ yolov5 설치
!pip install yolov5 -q  # ultralytics 패키지가 함께 설치됩니다.

# 2️⃣ 라이브러리 임포트 & 모델 로드
from yolov5 import YOLOv5
import cv2
import matplotlib.pyplot as plt

# Colab 런타임에 맞춰 device 선택 (cpu/gpu 자동)
model = YOLOv5("yolov5s.pt", device="")  # 가볍고 빠른 yolov5s 버전

# 3️⃣ 샘플 이미지 다운로드
!wget -q https://ultralytics.com/images/zidane.jpg -O sample.jpg
img = cv2.imread("sample.jpg")[:, :, ::-1]  # BGR→RGB

# 4️⃣ 추론 수행
results = model.predict(img)

# 5️⃣ 결과 시각화
annotated = results.render()[0]  # Annotated image
plt.figure(figsize=(8,5))
plt.imshow(annotated)
plt.axis("off")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


(np.float64(-0.5), np.float64(1279.5), np.float64(719.5), np.float64(-0.5))